# T3. dataloader 的内部结构和基本使用

&emsp; 1 &ensp; fastNLP 中的 dataloader
 
&emsp; &emsp; 1.1 &ensp; dataloader 的基本介绍

&emsp; &emsp; 1.2 &ensp; dataloader 的函数创建

&emsp; 2 &ensp; fastNLP 中 dataloader 的延伸

&emsp; &emsp; 2.1 &ensp; collator 的概念与使用

&emsp; &emsp; 2.2 &ensp; 结合 datasets 框架

## 1. fastNLP 中的 dataloader

### 1.1 dataloader 的基本介绍

在`fastNLP 1.0`的开发中，最关键的开发目标就是**实现 fastNLP 对当前主流机器学习框架**，例如

&emsp; **当下流行的 pytorch**，以及**国产的 paddle 、jittor 和 oneflow 的兼容**，扩大受众的同时，也是助力国产

本着分而治之的思想，我们可以将`fastNLP 1.0`对`pytorch`、`paddle`、`jittor`、`oneflow`框架的兼容，划分为

&emsp; &emsp; **对数据预处理**、**批量 batch 的划分与补齐**、**模型训练**、**模型评测**，**四个部分的兼容**

&emsp; 针对数据预处理，我们已经在`tutorial-1`中介绍了`dataset`和`vocabulary`的使用

&emsp; &emsp; 而结合`tutorial-0`，我们可以发现**数据预处理环节本质上是框架无关的**

&emsp; &emsp; 因为在不同框架下，读取的原始数据格式都差异不大，彼此也很容易转换

只有涉及到张量、模型，不同框架才展现出其各自的特色：**pytorch 和 oneflow 中的 tensor 和 nn.Module**

&emsp; &emsp; **在 paddle 中称为 tensor 和 nn.Layer**，**在 jittor 中则称为 Var 和 Module**

&emsp; &emsp; 因此，**模型训练、模型评测**，**是兼容的重难点**，我们将会在`tutorial-5`中详细介绍

&emsp; 针对批量`batch`的处理，作为`fastNLP 1.0`中框架无关部分想框架相关部分的过渡

&emsp; &emsp; 就是`dataloader`模块的职责，这也是本篇教程`tutorial-3`讲解的重点

**dataloader 模块的职责**，详细划分可以包含以下三部分，**采样划分、补零对齐、框架匹配**

&emsp; &emsp; 第一，确定`batch`大小，确定采样方式，划分后通过迭代器即可得到`batch`序列

&emsp; &emsp; 第二，对于序列处理，这也是`fastNLP`主要针对的，将同个`batch`内的数据对齐

&emsp; &emsp; 第三，**batch 内数据格式要匹配框架**，**但 batch 结构需保持一致**，**参数匹配机制**

&emsp; 对此，`fastNLP 1.0`给出了 **TorchDataLoader 、 PaddleDataLoader 、 JittorDataLoader 和 OneflowDataLoader**

&emsp; &emsp; 分别针对并匹配不同框架，但彼此之间参数名、属性、方法仍然类似，前两者大致如下表所示

名称|参数|属性|功能|内容
----|----|----|----|----|
 `dataset` | √ | √ | 指定`dataloader`的数据内容  |  |
 `batch_size` | √ | √ | 指定`dataloader`的`batch`大小 | 默认`16` |
 `shuffle` | √ | √ | 指定`dataloader`的数据是否打乱 | 默认`False` |
 `collate_fn` | √ | √ | 指定`dataloader`的`batch`打包方法 | 视框架而定 |
 `sampler` | √ | √ | 指定`dataloader`的`__len__`和`__iter__`函数的实现 | 默认`None` |
 `batch_sampler` | √ | √ | 指定`dataloader`的`__len__`和`__iter__`函数的实现 | 默认`None` |
 `drop_last` | √ | √ | 指定`dataloader`划分`batch`时是否丢弃剩余的 | 默认`False` |
 `cur_batch_indices` |  | √ | 记录`dataloader`当前遍历批量序号 |  |
 `num_workers` | √ | √ | 指定`dataloader`开启子进程数量 | 默认`0` |
 `worker_init_fn` | √ | √ | 指定`dataloader`子进程初始方法 | 默认`None` |
 `generator` | √ | √ | 指定`dataloader`子进程随机种子 | 默认`None` |
 `prefetch_factor` |  | √ | 指定为每个`worker`装载的`sampler`数量 | 默认`2` |

&emsp; 论及`dataloader`的函数，其中，`get_batch_indices`用来获取当前遍历到的`batch`序号，其他函数

&emsp; &emsp; 包括`set_ignore`、`set_pad`和`databundle`类似，请参考`tutorial-2`，此处不做更多介绍

&emsp; &emsp; 以下是`tutorial-2`中已经介绍过的数据预处理流程，接下来是对相关数据进行`dataloader`处理

In [1]:
import sys
sys.path.append('..')

import pandas as pd
from functools import partial
from fastNLP.transformers.torch import BertTokenizer

from fastNLP import DataSet
from fastNLP import Vocabulary
from fastNLP.io import DataBundle


class PipeDemo:
    def __init__(self, tokenizer='bert-base-uncased'):
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer)

    def process_from_file(self, path='./data/test4dataset.tsv'):
        datasets = DataSet.from_pandas(pd.read_csv(path, sep='\t'))
        train_ds, test_ds = datasets.split(ratio=0.7)
        train_ds, dev_ds = datasets.split(ratio=0.8)
        data_bundle = DataBundle(datasets={'train': train_ds, 'dev': dev_ds, 'test': test_ds})

        encode = partial(self.tokenizer.encode_plus, max_length=100, truncation=True,
                         return_attention_mask=True)
        data_bundle.apply_field_more(encode, field_name='Sentence', progress_bar='tqdm')
        
        target_vocab = Vocabulary(padding=None, unknown=None)

        target_vocab.from_dataset(*[ds for _, ds in data_bundle.iter_datasets()], field_name='Sentiment')
        target_vocab.index_dataset(*[ds for _, ds in data_bundle.iter_datasets()], field_name='Sentiment',
                                   new_field_name='target')

        data_bundle.set_pad('input_ids', pad_val=self.tokenizer.pad_token_id)
        data_bundle.set_ignore('SentenceId', 'Sentence', 'Sentiment')  
        return data_bundle

    
pipe = PipeDemo(tokenizer='bert-base-uncased')

data_bundle = pipe.process_from_file('./data/test4dataset.tsv')

print(data_bundle.get_dataset('train'))

[i 0604 15:44:29.773860 92 log.cc:351] Load log_sync: 1


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Processing:   0%|          | 0/2 [00:00<?, ?it/s]

Processing:   0%|          | 0/2 [00:00<?, ?it/s]

+------------+----------------+-----------+----------------+--------------------+--------------------+--------+
| SentenceId | Sentence       | Sentiment | input_ids      | token_type_ids     | attention_mask     | target |
+------------+----------------+-----------+----------------+--------------------+--------------------+--------+
| 1          | A series of... | negative  | [101, 1037,... | [0, 0, 0, 0, 0,... | [1, 1, 1, 1, 1,... | 1      |
| 4          | A positivel... | neutral   | [101, 1037,... | [0, 0, 0, 0, 0,... | [1, 1, 1, 1, 1,... | 2      |
| 3          | Even fans o... | negative  | [101, 2130,... | [0, 0, 0, 0, 0,... | [1, 1, 1, 1, 1,... | 1      |
| 5          | A comedy-dr... | positive  | [101, 1037,... | [0, 0, 0, 0, 0,... | [1, 1, 1, 1, 1,... | 0      |
+------------+----------------+-----------+----------------+--------------------+--------------------+--------+


### 1.2 dataloader 的函数创建

在`fastNLP 1.0`中，**更方便、可能更常用的 dataloader 创建方法是通过 prepare_xx_dataloader 函数**

&emsp; 例如下方的`prepare_torch_dataloader`函数，指定必要参数，读取数据集，生成对应`dataloader`

&emsp; 类型为`TorchDataLoader`，只能适用于`pytorch`框架，因此对应`trainer`初始化时`driver='torch'`

同时我们看还可以发现，在`fastNLP 1.0`中，**batch 表示为字典 dict 类型**，**key 值就是原先数据集中各个字段**

&emsp; **除去经过 DataBundle.set_ignore 函数隐去的部分**，而`value`值为`pytorch`框架对应的`torch.Tensor`类型

In [2]:
from fastNLP import prepare_torch_dataloader

train_dataset = data_bundle.get_dataset('train')
evaluate_dataset = data_bundle.get_dataset('dev')

train_dataloader = prepare_torch_dataloader(train_dataset, batch_size=16, shuffle=True)
evaluate_dataloader = prepare_torch_dataloader(evaluate_dataset, batch_size=16)

print(type(train_dataloader))

import pprint

for batch in train_dataloader:
    print(type(batch), type(batch['input_ids']), list(batch))
    pprint.pprint(batch, width=1)

<class 'fastNLP.core.dataloaders.torch_dataloader.fdl.TorchDataLoader'>
<class 'dict'> <class 'torch.Tensor'> ['input_ids', 'token_type_ids', 'attention_mask', 'target']
{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'input_ids': tensor([[  101,  1037,  4038,  1011,  3689,  1997,  3053,  8680, 19173, 15685,
          1999,  1037, 18006,  2836,  2011,  1996,  2516,  2839, 14996,  3054,
         15509,  5325,  1012,   1

之所以说`prepare_xx_dataloader`函数更方便，是因为其**导入对象不仅可也是 DataSet 类型**，**还可以**

&emsp; **是 DataBundle 类型**，不过数据集名称需要是`'train'`、`'dev'`、`'test'`供`fastNLP`识别

例如下方就是**直接通过 prepare_paddle_dataloader 函数生成基于 PaddleDataLoader 的字典**


In [3]:
from fastNLP import prepare_paddle_dataloader

dl_bundle = prepare_paddle_dataloader(data_bundle, batch_size=16, shuffle=True)

print(type(dl_bundle['train']))

<class 'fastNLP.core.dataloaders.paddle_dataloader.fdl.PaddleDataLoader'>


&emsp; 而在接下来`trainer`的初始化过程中，按如下方式使用即可，除了初始化时`driver='paddle'`外

&emsp; 这里也可以看出`trainer`模块中，**evaluate_dataloaders 的设计允许评测可以针对多个数据集**

```python
trainer = Trainer(
    model=model,
    train_dataloader=dl_bundle['train'],
    optimizers=optimizer,
	...
	driver='paddle',
	device='gpu',
	...
    evaluate_dataloaders={'dev': dl_bundle['dev'], 'test': dl_bundle['test']},     
    metrics={'acc': Accuracy()},
	...
)
```

## 2. fastNLP 中 dataloader 的延伸

### 2.1 collator 的概念与使用

在`fastNLP 1.0`中，在数据加载模块`dataloader`内部，如之前表格所列举的，还存在其他的一些模块

&emsp; 例如，**实现序列的补零对齐的核对器 collator 模块**；注：`collate vt. 整理（文件或书等）；核对，校勘`

在`fastNLP 1.0`中，虽然`dataloader`随框架不同，但`collator`模块却是统一的，主要属性、方法如下表所示

名称|属性|方法|功能|内容
 ----|----|----|----|----|
 `backend` | √ |  | 记录`collator`对应框架 | 字符串型，如`'torch'` |
 `padders` | √ |  | 记录各字段对应的`padder`，每个负责具体补零对齐&emsp; | 字典类型 |
 `ignore_fields` | √ |  | 记录`dataloader`采样`batch`时不予考虑的字段 | 集合类型 |
 `input_fields` | √ |  | 记录`collator`每个字段的补零值、数据类型等 | 字典类型 |
 `set_backend` |  | √ | 设置`collator`对应框架 | 字符串型，如`'torch'` |
 `set_ignore` |  | √ | 设置`dataloader`采样`batch`时不予考虑的字段 | 字符串型，表示`field_name`&emsp; |
 `set_pad` |  | √ | 设置`collator`每个字段的补零值、数据类型等 |  |

In [4]:
train_dataloader.collate_fn

print(type(train_dataloader.collate_fn))

<class 'function'>


此外，还可以 **手动定义 dataloader 中的 collate_fn**，而不是使用`fastNLP 1.0`中自带的`collator`模块

&emsp; 该函数的定义可以大致如下，需要注意的是，**定义 collate_fn 之前需要了解 batch 作为字典的格式**

&emsp; 该函数通过`collate_fn`参数传入`dataloader`，**在 batch 分发**（**而不是 batch 划分**）**时调用**

In [5]:
import torch

def collate_fn(batch):
    input_ids, atten_mask, labels = [], [], []
    max_length = [0] * 3
    for each_item in batch:
        input_ids.append(each_item['input_ids'])
        max_length[0] = max(len(each_item['input_ids']), max_length[0])
        atten_mask.append(each_item['token_type_ids'])
        max_length[1] = max(len(each_item['token_type_ids']), max_length[1])
        labels.append(each_item['attention_mask'])
        max_length[2] = max(len(each_item['attention_mask']), max_length[2])

    for i in range(3):
        each = (input_ids, atten_mask, labels)[i]
        for item in each:
            item.extend([0] * (max_length[i] - len(item)))
    return {'input_ids': torch.cat([torch.tensor([item]) for item in input_ids], dim=0),
            'token_type_ids': torch.cat([torch.tensor([item]) for item in atten_mask], dim=0),
            'attention_mask': torch.cat([torch.tensor(item) for item in labels], dim=0)}

注意：使用自定义的`collate_fn`函数，`trainer`的`collate_fn`变量也会自动调整为`function`类型

In [6]:
train_dataloader = prepare_torch_dataloader(train_dataset, collate_fn=collate_fn, shuffle=True)
evaluate_dataloader = prepare_torch_dataloader(evaluate_dataset, collate_fn=collate_fn, shuffle=True)

print(type(train_dataloader))
print(type(train_dataloader.collate_fn))

for batch in train_dataloader:
    pprint.pprint(batch, width=1)

<class 'fastNLP.core.dataloaders.torch_dataloader.fdl.TorchDataLoader'>
<class 'function'>
{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([[  101,  1037,  4038,  1011,  3689,  1997,  3053,  8680, 19173, 15685,
          1999,  1037, 18006,  2836,  2011,  1996,  2516,  2839, 14996,  3054,
         15509,  5325,  1012,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,  

### 2.2  fastNLP 与 datasets 的结合

从`tutorial-1`至`tutorial-3`，我们已经完成了对`fastNLP v1.0`数据读取、预处理、加载，整个流程的介绍

&emsp; 不过在实际使用中，我们往往也会采取更为简便的方法读取数据，例如使用`huggingface`的`datasets`模块

**使用 datasets 模块中的 load_dataset 函数**，通过指定数据集两级的名称，示例中即是**GLUE 标准中的 SST-2 数据集**

&emsp; 即可以快速从网上下载好`SST-2`数据集读入，之后以`pandas.DataFrame`作为中介，再转化成`fastNLP.DataSet`

&emsp; 之后的步骤就和其他关于`dataset`、`databundle`、`vocabulary`、`dataloader`中介绍的相关使用相同了

In [7]:
from datasets import load_dataset

sst2data = load_dataset('glue', 'sst2')

dataset = DataSet.from_pandas(sst2data['train'].to_pandas())

Reusing dataset glue (/remote-home/xrliu/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]